<a href="https://colab.research.google.com/github/joaomerjam/ECON3916-33674-Statistical-Machine-Learning/blob/main/Lab6/Lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

In [51]:
# getting data
df = sns.load_dataset('titanic')
#df.head()

print(f"Total Population: {len(df)}")
print(f"Population Survival Rate: {df['survived'].mean():.4f}")

df.describe()


Total Population: 891
Population Survival Rate: 0.3838


,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Class Practice

In [36]:
# class demonstration
index_test = np.random.permutation([0,1,2,3,4,5,6,7,8,9])
index_test[8:]

array([1, 4])

In [37]:
int(len(index_test)*0.8)

8

In [40]:
index_test[int(len(index_test)*0.8)]

np.int64(1)

###Lab Continued

In [52]:
#shuffle
np.random.seed(2026)
indicies = np.random.permutation(df.index)

In [54]:
# cut the deck (80/20 Split)
split_point = int(len(df) *0.8)
 # split_point gives 712
# slicing the shuffled indices
train_idx = indicies[:split_point]
test_idx = indicies[split_point:]

# creating the subsets
train_set = df.loc[train_idx]
test_set = df.loc[test_idx]

# bias Check (The Delta)
train_surv = train_set['survived'].mean()
test_surv = test_set['survived'].mean()
delta = abs(train_surv - test_surv)

print(f"Train Survival Rate: {train_surv:.4f}")
print(f"Test Survival Rate:  {test_surv:.4f}")
print(f"Delta:               {delta:.4f}")

Train Survival Rate: 0.3736
Test Survival Rate:  0.4246
Delta:               0.0510


In [55]:
df.corr(numeric_only=True)

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307,-0.557080,-0.203367
pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500,0.094035,0.135207
age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067,0.280328,0.198270
sibsp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651,-0.253586,-0.584471
parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225,-0.349943,-0.583398
fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000,-0.182024,-0.271832
adult_male,-0.557080,0.094035,0.280328,-0.253586,-0.349943,-0.182024,1.000000,0.404744
alone,-0.203367,0.135207,0.198270,-0.584471,-0.583398,-0.271832,0.404744,1.000000


Pclass is at -0.333: depending on your class, your survival rate can be lower.
adult_male is at -0.55: stronger negative correlation. If you are male, less likely to survive.

Adult Male is binary data, so less detail to prove selection bias. So, Pclass is better for this, more based on intution.

In [57]:
from sklearn.model_selection import train_test_split

# stratify by 'pclass' ensures the distribution of classes is identical
X_train, X_test = train_test_split(df, test_size=0.2, stratify=df['pclass'],random_state=42)

print("\n--- Stratified Split ---")
print("Train Class Dist:\n", X_train['pclass'].value_counts(normalize=True))
print("Test Class Dist:\n", X_test['pclass'].value_counts(normalize=True))


--- Stratified Split ---
Train Class Dist:
 pclass
3    0.550562
1    0.242978
2    0.206461
Name: proportion, dtype: float64
Test Class Dist:
 pclass
3    0.553073
1    0.240223
2    0.206704
Name: proportion, dtype: float64


### Step 4
Ai Pormpt:

"I need to perform a forensic check on an A/B test using Python.

Context:
I ran an experiment on 1000 users with a planned 50/50 split.
However, I observed 450 users in Control and 550 in Treatment.

Please write Python code using 'scipy.stats.chisquare' to:
1. Define the observed and expected arrays.
2. Calculate the Chi-Square statistic and the p-value.
3. Print a conclusion:
   - If p < 0.01, print 'CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.'
   - Else, print 'Variance is within natural limits.'

Explain to me briefly why a 550/450 split is not just 'bad luck'."



In [58]:
import numpy as np
from scipy.stats import chisquare

# ── 1. Define observed vs. expected counts ──────────────────────────
observed = np.array([450, 550])   # [Control, Treatment]
expected = np.array([500, 500])   # planned 50/50 split

# ── 2. Chi-Square goodness-of-fit test ──────────────────────────────
chi2_stat, p_value = chisquare(f_obs=observed, f_exp=expected)

print(f"Observed:        Control = {observed[0]}, Treatment = {observed[1]}")
print(f"Expected:        Control = {expected[0]}, Treatment = {expected[1]}")
print(f"Chi² Statistic:  {chi2_stat:.4f}")
print(f"P-value:         {p_value:.6f}")
print("-" * 55)

# ── 3. Conclusion ──────────────────────────────────────────────────
if p_value < 0.01:
    print("CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. "
          "Check Load Balancer.")
else:
    print("Variance is within natural limits.")

Observed:        Control = 450, Treatment = 550
Expected:        Control = 500, Treatment = 500
Chi² Statistic:  10.0000
P-value:         0.001565
-------------------------------------------------------
CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.
